<a href="https://colab.research.google.com/github/zyang63/Die_casting_ejection/blob/main/solidification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geometric Analysis in Google Colab


The code installs and imports several libraries for different purposes. The trimesh library is installed to work with triangular meshes, SimpleITK is installed for medical image analysis, scipy is installed for scientific computing, and bpy is installed for 3D modeling and rendering. The directory "geometry" is created to store the various stl files generated by the analysis. The result of the code is stored in the base directory and produces three "gltf" animations which can be downloaded and reviewed.

In [1]:
%%capture
try:
  import open3d as o3d
except ImportError:
  !pip install open3d
  import open3d as o3d
try:
  import SimpleITK as sitk
except ImportError:
  !pip install SimpleITK
  import SimpleITK as sitk
try:
  import bpy
except ImportError:
  !pip install bpy
  import bpy
import numpy as np
from scipy import ndimage
from scipy.special import erf
from skimage import measure
import os

#Setup

In [2]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
#@title #File Entry { display-mode: "form"}
#@markdown User can choose to upload the file to colab directly or select the google file upload button at the bottom of this form. Also choose the number of elements.

button_pressed = False  # Initialize the variable as False
filename = ""
button = widgets.Button(description="Google upload dialog")
output = widgets.Output()

def on_button_clicked(b):
    global button_pressed  # Access the global variable
    global filename
    with output:
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        button_pressed = True  # Set the variable to True when the button is clicked

button.on_click(on_button_clicked)
display(button, output)

Button(description='Google upload dialog', style=ButtonStyle())

Output()

In [83]:
#@title #Element Count { display-mode: "form", run: "auto" }
element_count = 200 #@param {type:"slider", min:10, max:500, step:1}
if not button_pressed:
  filename = "/content/1.stl" #@param {type:"string"}
geometry = o3d.io.read_triangle_mesh(filename)
voxel_resolution = (geometry.get_max_bound()-geometry.get_min_bound()).max()/ element_count
print("File used ", filename)
print("Element size is in units from stl file ",voxel_resolution, " per cell")

File used  /content/1.stl
Element size is in units from stl file  0.01  per cell


# Voxelizing the geometry


The following code performs voxelization and mesh conversion operations. It starts by voxelizing a mesh with a specified voxel size. Then, it creates a solid voxel representation by filling the voxel surface. The solid voxel data is padded with two layers of zeros. The padded voxel data is further processed using marching cubes algorithm to obtain a mesh representation. Finally, the resulting mesh is exported as an STL file named "marching_geometry.stl" in the geometry directory.

In [84]:
pcd = geometry.sample_points_uniformly(number_of_points=100000000)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,voxel_size=voxel_resolution)
voxels = voxel_grid.get_voxels()
indices = np.stack(list(vx.grid_index for vx in voxels))
del voxels, voxel_grid, pcd
max_indices = np.max(indices, axis=0)+1
dense_array = np.zeros(max_indices, dtype=np.bool8)
for idx in indices:
    dense_array[tuple(idx)] = 1
del indices
array_pad = np.pad(dense_array.astype(bool),((2,2)),'constant')
del dense_array
array_closing = ndimage.binary_closing(array_pad, structure=ndimage.generate_binary_structure(3, 1), iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
img = sitk.GetImageFromArray(array_closing.astype(int))
seg = sitk.ConnectedComponent(img != img[0,0,0])
img_filled = sitk.BinaryFillhole(seg!=0)
array_filled = sitk.GetArrayFromImage(img_filled)
verts, faces, _, _ = measure.marching_cubes(array_filled)
vertices_original = np.asarray(verts.tolist())
faces_original = np.asarray(faces.tolist())
del array_closing, img, seg, img_filled

<ipython-input-84-e7cb7dcff956>:7: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  dense_array = np.zeros(max_indices, dtype=np.bool8)


# Morphological operations and Distance transform on mesh


The following code applies binary closing to a padded voxel data array and assigns the result to array_initial for further use. It then defines a function named core that performs operations to extract the core of a structure from an input array. The function is applied to array_initial, and the resulting array is assigned to array_core. The code further manipulates array_core by setting non-core elements to 0 and core elements to 1. Overall, these operations help identify and isolate the core of a structure within the voxel data.

In [85]:
array_initial = ndimage.binary_closing(array_filled, structure=ndimage.generate_binary_structure(3, 1), iterations=1, mask=None,  border_value=0, origin=0, brute_force=False)
img_initial = sitk.GetImageFromArray(array_initial.astype(int))
def core(array):
    directions = [(0, 0, 0, 0), (2, 1, 2, 1), (0, 3, 2, 0)]
    new_array = np.copy(array)
    for direction in directions:
      array = np.rollaxis(array, direction[0], direction[1])
      array_temp = np.copy(array)
      ones_indices = np.argwhere(array == 1)
      for (i, j, start_index), (next_i, next_j, end_index) in zip(ones_indices[:-1], ones_indices[1:]):
          if i == next_i and j == next_j:
              array_temp[i, j, start_index+1:end_index] = 2
      array_temp = np.rollaxis(array_temp, direction[2], direction[3])
      array = np.rollaxis(array, direction[2], direction[3])
      new_array = np.where((array_temp == 2) | (new_array == 2), 2, array)
    return new_array
array_core = core(array_initial.astype(int))
array_core[array_core != 2] = 0
array_core[array_core == 2] = 1

## Distance Field Calculation
The following code calculates the signed Maurer distance map from a binary voxel data representation. The distance map is computed using the SignedMaurerDistanceMap function from the SimpleITK library. The resulting distance map is then converted into a NumPy array. The code further determines the depth of the object by taking the negative minimum value from the distance map and converting it to an integer. This depth value represents the distance inside the object. Overall, these operations provide information about the spatial distribution and depth of the object in the voxel data.

In [86]:
img_dist = sitk.SignedMaurerDistanceMap(img_initial != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False)
array_dist = sitk.GetArrayFromImage(img_dist)
depth = int(-array_dist.min())
print(depth)
del img_dist, img_initial

5


## Solidification

# geometry

In [87]:
A = 0.2866
T_0 = 298.15
T_m = 933

In [88]:
C = erf(A)

In [89]:
T_s = (T_0*10000*C + T_m*25330)/(10000*C + 25330)
print("The surface temperature is:", T_s)

The surface temperature is: 862.8320680299192


In [90]:
heat_transfer = np.zeros_like(array_dist)
for i in range(1, depth+1):
    thick = np.where(array_dist > -i, 0, 1)
    array_thick_dilate = ndimage.binary_dilation(thick, structure=ndimage.generate_binary_structure(3, 1), iterations=i, mask=None, border_value=0, origin=0, brute_force=False)
    heat_transfer += array_thick_dilate.astype(int)
heat_transfer[array_filled == 0] = 0
heat_transfer_array = np.array(heat_transfer)

In [91]:
solidification_time = np.sqrt((heat_transfer_array * voxel_resolution)/0.57)
solidification_time_array = np.array(solidification_time)
arr_no_zeros = np.where(solidification_time_array == 0, np.inf, solidification_time_array)
min_value = np.min(arr_no_zeros)
time_depth = (solidification_time.max() - min_value) / depth
print(time_depth)
time_depth_change = solidification_time.max() - min_value
print("The solidification time is:", solidification_time_array.max())

0.03274424076080322
The solidification time is: 0.29617444


In [92]:
bpy.ops.wm.read_factory_settings(use_empty=True)
for i in np.arange(0,depth+1):
  start_range = min_value + i * time_depth
  end_range = min_value + (i + 1) * time_depth

    # Create a mask for elements within the range
  mask = (solidification_time >= start_range) & (solidification_time < end_range)
  print(f"For the range {start_range} to {end_range}:")

  if mask.any():
        shape = np.where(mask, solidification_time, 0)
        print(shape.max())
  verts, faces, _, _ = measure.marching_cubes(shape)
  vertices = np.asarray(verts.tolist())
  faces = np.asarray(faces.tolist())
  blender_mesh = bpy.data.meshes.new("Mesh")
  blender_mesh.from_pydata(vertices, [], faces)

# Create mesh object
  blender_object = bpy.data.objects.new("Object", blender_mesh)
  scene = bpy.context.scene
  scene.collection.objects.link(blender_object)

# Select all vertices and dissolve small faces
  bpy.context.view_layer.objects.active = blender_object
  bpy.ops.object.mode_set(mode='EDIT')
  bpy.ops.mesh.select_all(action='SELECT')
  bpy.ops.mesh.dissolve_limited(angle_limit=0.1)  # Adjust the angle limit as needed
  bpy.ops.object.mode_set(mode='OBJECT')
  obj = bpy.data.objects.new(f"Object{i}", blender_mesh)
  bpy.context.collection.objects.link(obj)
  obj.select_set(True)
  bpy.context.view_layer.objects.active = obj
  #bpy.ops.object.modifier_add(type="DECIMATE")
  #bpy.context.object.modifiers["Decimate"].decimate_type = "COLLAPSE"
  #bpy.context.object.modifiers["Decimate"].ratio = 0.1
  #bpy.ops.object.modifier_apply(modifier="Decimate")
  obj.color = (0,0,1,1)
  mat = bpy.data.materials.new(f"{i}")
  mat.use_nodes = True
  principled = mat.node_tree.nodes["Principled BSDF"]
  principled.inputs["Base Color"].default_value = (i * time_depth/(solidification_time.max() - min_value),0,1-(i * time_depth/(solidification_time.max() - min_value)),1)
  obj.data.materials.append(mat)
  bpy.ops.export_scene.gltf(filepath="/content/geometry_solidification_time_color.glb")

For the range 0.13245323300361633 to 0.16519747376441957:
0.13245323
01:27:11 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
01:27:11 | INFO: Starting glTF 2.0 export
01:27:11 | INFO: Extracting primitive: Mesh
01:27:11 | INFO: Primitives created: 1
01:27:11 | INFO: Finished glTF 2.0 export in 0.010061264038085938 s

For the range 0.16519747376441957 to 0.19794171452522277:
0.18731716
01:27:11 | ERROR: Draco mesh compression is not available because library could not be found at /content/4.0/python/lib/python3.10/site-packages/libextern_draco.so
01:27:11 | INFO: Starting glTF 2.0 export
01:27:11 | INFO: Extracting primitive: Mesh
01:27:11 | INFO: Primitives created: 1
01:27:11 | INFO: Extracting primitive: Mesh.001
01:27:11 | INFO: Primitives created: 1
01:27:11 | INFO: Finished glTF 2.0 export in 0.01913595199584961 s

For the range 0.19794171452522277 to 0.23068595528602598:
0.